# Recomendador de artigos - OASIS

## Importação das bibliotecas

In [1]:
from bs4 import BeautifulSoup
from urllib.request import urlopen, Request
from urllib.request import URLError, HTTPError
import pandas as pd
from IPython.display import clear_output


pd.options.display.max_columns = 999
pd.options.display.max_info_rows = 999

In [65]:
import pandas as pd
import os
import requests

## Funções

In [2]:
def resposta_html(url,headers):
    '''
    Esta função recebe uma URL como entrada, faz uma solicitação HTTP usando o cabeçalho armazenado na variável de instância headers, 
    lê a resposta HTTP e retorna o HTML da resposta.
    '''
    try: 
        req = Request(url, headers= headers)
        response = urlopen(req)
        html = response.read()
    except HTTPError as e:
        print(e.status, e.reason)
    except URLError as e:
        print(e.reason)
    return html


def obtem_total_paginas(html):
    '''
    Esta função recebe o HTML da página de resultados de pesquisa e retorna o número total de páginas 
    de resultados que correspondem à pesquisa. 
    '''
    soup = BeautifulSoup(html, 'html.parser')
    total_paginas = soup.find('ul', {'class': 'pagination'}).find_all('li')[-1].get_text()
    total_paginas = total_paginas.replace('[', '').replace(']', '')
    total_paginas = int(total_paginas)
    return total_paginas


def links_paginas(url, total_paginas,headers):
    '''
    Esta função recebe uma URL de pesquisa e o número total de páginas de resultados como entrada. 
    Em seguida, extrai os links para todas as páginas de resultados e retorna uma lista de links para cada página.
    '''
    links_paginas = []
    numero_pagina = 1
    while numero_pagina < total_paginas + 1:
        numero_pagina = str(numero_pagina)
        resposta_html_pagina = resposta_html(url + f'&page={numero_pagina}',headers=headers)
        soup = BeautifulSoup(resposta_html_pagina, 'html.parser')
        for card in soup.findAll('div', {'class': 'result'}):
            links_paginas.append('https://oasisbr.ibict.br' + card.find('a', {'class': 'title getFull'}).get('href'))
        numero_pagina = int(numero_pagina)
        numero_pagina += 1
        
        print(f'Coletando página {numero_pagina} de {total_paginas}')
        clear_output(wait=True)
        
    return links_paginas


def scraping_dados(links_paginas,headers):
    '''
    A função extrai informações como título, autor, data de publicação e tipo de documento para cada página e armazena 
    as informações em uma lista.
    '''
    dados_lista = []
    
    for idx,link in enumerate(links_paginas):
        url = link
        html = resposta_html(url,headers=headers)
        soup = BeautifulSoup(html,'html.parser')

        table = soup.find('table',{'class':'citation table table-striped'})

        data = {}
        for row in table.find_all('tr'):
            th = row.find('th').get_text()
            td = row.find('td').get_text()
            data[th] = td
        dados_lista.append(data)
        
        print(f'Coletando link {idx} de {len(links_paginas)}')
        clear_output(wait=True)
            

    return dados_lista


def dataframe_dados_lista(lista_dados):
    '''
    Esta função transforma uma lista de dicionários com os dados coletados da raspagem e retorna um DataFrame do pandas com esses dados.
    
    '''
    
    df = pd.DataFrame(lista_dados)[['title','description','url','eu_rights_str_mv','network_name_str','topic']]

    df['title'] = df['title'].str.strip().str.replace('\n                  ','')
    df['description'] = df['description'].str.strip().str.replace('\n                  ','')
    df['url'] = df['url'].str.replace('\n                  ','').str.replace('\n','')
    df['eu_rights_str_mv'] = df['eu_rights_str_mv'].str.replace('\n                  ','').str.replace('\n','')
    df['network_name_str'] = df['network_name_str'].str.replace('\n                  ','').str.replace('\n','')
    df['topic'] = df['topic'].str.strip().str.replace('.','').str.split('\n                  ').apply(lambda x: ';;;'.join(x) if isinstance(x, list) else x)

    return df

## Coleta de dados

In [ ]:
#url = 'https://oasisbr.ibict.br/vufind/Search/Results?filter%5B%5D=%7Eformat%3A%22masterThesis%22&filter%5B%5D=%7Eformat%3A%22doctoralThesis%22&type=AllFields&daterange%5B%5D=publishDate&publishDatefrom=2023&publishDateto=2023'
#headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36'}

#resposta = resposta_html(url,headers)

#total_paginas = obtem_total_paginas(resposta)

#links = links_paginas(url,total_paginas,headers)

In [ ]:
#lista_dados = scraping_dados(links,headers)

#df = dataframe_dados_lista(lista_dados)

#df.to_csv('teses_dissertacoes_oasis_2023_20231005.csv',sep=';',index=False)


## Transformações dos dados

In [47]:
df = pd.read_csv('teses_dissertacoes_oasis_2023_20231005.csv',sep=';')

In [48]:
df

,title,description,url,eu_rights_str_mv,network_name_str,topic
0,Análise do conteúdo de ecologia nos livros de ...,Dissertação (mestrado) — Universidade de Brasí...,https://repositorio.unb.br/handle/10482/45512,openAccess,Repositório Institucional da UnB,Ensino investigativo;;;Biologia - estudo e ens...
1,Distopia brasilis: o mito de Narciso e Eco nos...,The work proposes to examine racist discourse ...,http://tede2.unicap.br:8080/handle/tede/1685,openAccess,Biblioteca Digital de Teses e Dissertações da ...,Dissertações;;;Ato (Filosofia);;;Linguagem - F...
2,A aposentadoria se aproxima: e agora ?: percep...,This research had as general objective to unde...,http://tede2.unicap.br:8080/handle/tede/1692,openAccess,Biblioteca Digital de Teses e Dissertações da ...,Dissertações;;;Envelhecimento;;;Aposentadoria ...
3,Habilidades pragmáticas em crianças com transt...,The year 2020 started with many challenges. In...,http://tede2.unicap.br:8080/handle/tede/1700,openAccess,Biblioteca Digital de Teses e Dissertações da ...,Dissertações;;;Pragmática;;;Autismo em criança...
4,Correção de textos escolares mediada por ferra...,The feedback practice in the school context ca...,http://tede2.unicap.br:8080/handle/tede/1701,openAccess,Biblioteca Digital de Teses e Dissertações da ...,Teses;;;Escrita - Estudo e ensino;;;Lingüístic...
...,...,...,...,...,...,...
14891,As Vivências e o deslocamento de refugiados ve...,A LGBTIfobia atinge a comunidade LGBTI+ ao red...,http://repositorio.ufgd.edu.br/jspui/handle/pr...,openAccess,Biblioteca Digital de Teses e Dissertações da ...,CNPQ::OUTROS::SOCIAIS E HUMANIDADES;;;Refugiad...
14892,Caracterização da cinza do bagaço da fécula de...,A crescente conscientização sobre a necessidad...,http://repositorio.ufgd.edu.br/jspui/handle/pr...,openAccess,Biblioteca Digital de Teses e Dissertações da ...,CNPQ::CIENCIAS EXATAS E DA TERRA;;;Resíduo agr...
14893,A Luta pela autogestão do serviço de saúde ind...,Essa dissertação de mestrado tem como objetivo...,http://repositorio.ufgd.edu.br/jspui/handle/pr...,openAccess,Biblioteca Digital de Teses e Dissertações da ...,CNPQ::CIENCIAS DA SAUDE::SAUDE COLETIVA::SAUDE...
14894,O Papel do lugar na geografia escolar: uma aná...,Este estudo constitui uma análise do papel do ...,http://repositorio.ufgd.edu.br/jspui/handle/pr...,openAccess,Biblioteca Digital de Teses e Dissertações da ...,CNPQ::CIENCIAS HUMANAS::GEOGRAFIA::GEOGRAFIA H...


In [49]:
df['qtd_termos'] = df['topic'].str.split(';;;').apply(lambda x: len(x) if isinstance(x,list) else 0)

In [50]:
df.qtd_termos.value_counts()

qtd_termos
9     1521
10    1459
5     1378
2     1353
11    1263
8     1258
6     1224
7     1182
4     1026
3      753
12     595
13     522
14     288
15     274
1      246
16     180
17     108
18      76
19      46
21      34
20      31
0       26
22      14
23      11
25      10
24       6
27       4
26       3
30       2
33       1
29       1
42       1
Name: count, dtype: int64

In [51]:
df.qtd_termos.value_counts(normalize=True) * 100

qtd_termos
9     10.210795
10     9.794576
5      9.250806
2      9.082975
11     8.478786
8      8.445220
6      8.216971
7      7.935016
4      6.887755
3      5.055048
12     3.994361
13     3.504296
14     1.933405
15     1.839420
1      1.651450
16     1.208378
17     0.725027
18     0.510204
19     0.308808
21     0.228249
20     0.208110
0      0.174544
22     0.093985
23     0.073845
25     0.067132
24     0.040279
27     0.026853
26     0.020140
30     0.013426
33     0.006713
29     0.006713
42     0.006713
Name: proportion, dtype: float64

In [52]:
(df.qtd_termos.value_counts(normalize=True) * 100).reset_index().query('3 < qtd_termos < 10').proportion.sum()

50.94656283566058

In [53]:
df_filtrado = df.query('3 < qtd_termos < 10')

In [ ]:
df_filtrado

In [55]:
df_filtrado['topic']

2        Dissertações;;;Envelhecimento;;;Aposentadoria ...
8        Dissertações;;;Aquisição de linguagem;;;Autism...
9        Teses;;;Direito do trabalho;;;Trabalhadores;;;...
12       Metodologia ativa;;;Aprendizagem baseada em pr...
15       muro de contenção;;;internet das coisas;;;moni...
                               ...                        
14891    CNPQ::OUTROS::SOCIAIS E HUMANIDADES;;;Refugiad...
14892    CNPQ::CIENCIAS EXATAS E DA TERRA;;;Resíduo agr...
14893    CNPQ::CIENCIAS DA SAUDE::SAUDE COLETIVA::SAUDE...
14894    CNPQ::CIENCIAS HUMANAS::GEOGRAFIA::GEOGRAFIA H...
14895    CNPQ::CIENCIAS HUMANAS::GEOGRAFIA;;;Cultura un...
Name: topic, Length: 7589, dtype: object

In [56]:
#Verificar termos do "topic"
for i in df_filtrado['topic']:
    print(i)

Dissertações;;;Envelhecimento;;;Aposentadoria - Aspectos psicológicos;;;Aposentadoria - Servidores públicos;;;Dissertations;;;Aging;;;Retirement - Psychological Aspects;;;Retirement - Civil Servants;;;CIENCIAS HUMANAS::PSICOLOGIA
Dissertações;;;Aquisição de linguagem;;;Autismo em crianças;;;Multimodalidade (Linguística);;;Dissertations;;;Language acquisition;;;Autism in children;;;Multimodality (Linguistics);;;LINGUISTICA, LETRAS E ARTES
Teses;;;Direito do trabalho;;;Trabalhadores;;;Plataformas digitais de trabalho - Regulamentação;;;Theses;;;Labor law;;;Workers;;;Digital work platforms - Regulation;;;CIENCIAS SOCIAIS APLICADAS::DIREITO
Metodologia ativa;;;Aprendizagem baseada em problema;;;Ética;;;Ensino jurídico;;;Active methodology;;;Problem based learning;;;Ethic;;;Legal education;;;CIENCIAS HUMANAS::EDUCACAO
muro de contenção;;;internet das coisas;;;monitoramento de integridade estrutural;;;nuvem;;;retaining wall;;;internet of things;;;structural health monitoring;;;cloud;;;CIENCI

In [57]:
df_filtrado_topics = df_filtrado.loc[:,['topic']].copy()

In [58]:
df_filtrado = df_filtrado_topics.copy()
df_filtrado['topic_formatado'] = df_filtrado['topic'].apply(lambda x: ';;;'.join([termo.strip() for termo in x.split(';;;')]))
df_filtrado['topic_formatado'] = df_filtrado['topic_formatado'].str.upper()

In [59]:
for i in df_filtrado['topic_formatado'].str.split(';;;'):
    print(i)

['DISSERTAÇÕES', 'ENVELHECIMENTO', 'APOSENTADORIA - ASPECTOS PSICOLÓGICOS', 'APOSENTADORIA - SERVIDORES PÚBLICOS', 'DISSERTATIONS', 'AGING', 'RETIREMENT - PSYCHOLOGICAL ASPECTS', 'RETIREMENT - CIVIL SERVANTS', 'CIENCIAS HUMANAS::PSICOLOGIA']
['DISSERTAÇÕES', 'AQUISIÇÃO DE LINGUAGEM', 'AUTISMO EM CRIANÇAS', 'MULTIMODALIDADE (LINGUÍSTICA)', 'DISSERTATIONS', 'LANGUAGE ACQUISITION', 'AUTISM IN CHILDREN', 'MULTIMODALITY (LINGUISTICS)', 'LINGUISTICA, LETRAS E ARTES']
['TESES', 'DIREITO DO TRABALHO', 'TRABALHADORES', 'PLATAFORMAS DIGITAIS DE TRABALHO - REGULAMENTAÇÃO', 'THESES', 'LABOR LAW', 'WORKERS', 'DIGITAL WORK PLATFORMS - REGULATION', 'CIENCIAS SOCIAIS APLICADAS::DIREITO']
['METODOLOGIA ATIVA', 'APRENDIZAGEM BASEADA EM PROBLEMA', 'ÉTICA', 'ENSINO JURÍDICO', 'ACTIVE METHODOLOGY', 'PROBLEM BASED LEARNING', 'ETHIC', 'LEGAL EDUCATION', 'CIENCIAS HUMANAS::EDUCACAO']
['MURO DE CONTENÇÃO', 'INTERNET DAS COISAS', 'MONITORAMENTO DE INTEGRIDADE ESTRUTURAL', 'NUVEM', 'RETAINING WALL', 'INTERNET OF

## Classificar termos únicos

In [37]:
set_termos_unicos = set()

for row in df_filtrado['topic_formatado']:
    for termo in row.split(';;;'):
        set_termos_unicos.add(termo)

lista_termos_unicos = list(set_termos_unicos)

In [60]:
lista_termos_unicos = sorted(lista_termos_unicos)

In [61]:
len(lista_termos_unicos)

30903

In [63]:
30903 / 100

309.03

In [64]:
batch_size = 100
delimiter = '-' * 100

for i in range(0, len(lista_termos_unicos), batch_size):
    batch = lista_termos_unicos[i:i + batch_size]
    batch_number = i // batch_size + 1
    print(40*'+',f"LOTE {batch_number}",40*'+')
    print('\n'.join(batch))
    print(delimiter,'\n\n\n\n\n\n\n\n')



++++++++++++++++++++++++++++++++++++++++ LOTE 1 ++++++++++++++++++++++++++++++++++++++++

"SEASCAPES"
(AUTO)BIOGRAFIA
(HETERO)ARENOS ATIVADOS
(IN)DISPONIBILIDADE EMOCIONAL DOS PAIS
(Q)SARS
(RE) EXISTÊNCIAS
-ENFERMAGEM PEDIÁTRICA
-PROMOÇÃO DA SAÚDE
/S/ EM CODA SILÁBICA
/S/ IN SYLLABLE CODA POSITION
00354
1 ST CEB
1 ST CYCLE OF BASIC EDUCATION
1,3- BENZODIOXOLA
1,3-DIARYLTRIAZENES
13 REASONS WHY
1325 RESOLUTION
15-MINUTE CITY
15-MINUTES CITY
18-GLICOSIL-HIDROLASES
1968
19TH CENTURY
19TH CENTURY PHOTOGRAPHY
1ST CYCLE
1ST CYCLE OF BASIC EDUCATION
1º CEB
1º CICLO
1º CICLO DO ENSINO BÁSICO
1º GRAU
2007 FINANCIAL CRISIS
20TH CENTURY
20TH CENTURY FRENCH LITERATURE
22Q112
2D
2D POSES
2D SYSTEM
2DEG
2S2P1D
2º CICLO DO ENSINO BÁSICO
3D
3D AUDIO MIXING
3D LOCALIZATION AND MAPPING
3D MODELING
3D PRINT
3D PRINT DATASET
3D PRINTING
3D SCANNING
3D SOFTWARES
3DPMD
3PL
40 INDUSTRY
5'-NUCLEOTIDASE
5'S METHODOLOGY
5-FLUOROURACIL
51535
515353
51539
5196
5G
5S
5S METHODOLOGY
5TH GRADE
5º ANO DE ESCOLARIDADE

In [ ]:
termos_classificados = {


}

# Revendo Open Alex

In [66]:
def coletar_dados_e_salvar(start_date, end_date):
    
    cursor = '*'
    
        
    contador = 1
    contador_erro = 0
    
    if not os.path.exists(f'datasets_{start_date}_to_{end_date}'):
        os.makedirs(f'datasets_{start_date}_to_{end_date}')
    
    while cursor != None:
        url = f'https://api.openalex.org/works?filter=from_publication_date:{start_date},to_publication_date:{end_date},type:article&per-page=200&cursor={cursor}'

        
        try:
            requisicao = requests.get(url)
            pagina_com_resultados = requisicao.json()
            
        except (requests.exceptions.RequestException, ValueError) as e:
            contador_erro += 1 
            print(f"Erro na página {contador} (Erro {contador_erro}): {e}")
            continue 
        
        resultados = pagina_com_resultados.get('results', [])
        
        df = pd.DataFrame(resultados)
        parquet_arquivo = os.path.join(f'datasets_{start_date}_to_{end_date}', f'registros_{contador}.parquet')
        df.to_parquet(parquet_arquivo, index=False)
        
        cursor = pagina_com_resultados['meta'].get('next_cursor')
        
        contador += 1
    
    print(f"Total de erros: {contador_erro}")

In [70]:
def extrair_concepts_scores(df):
    concept_data = []

    for concepts in df['concepts']:
        concept_scores = {}
        for concept in concepts:
            if concept['level'] == 0:
                concept_scores[concept['display_name']] = concept['score']
        concept_data.append(concept_scores)

    df_concepts = pd.DataFrame(concept_data, index=df.index).fillna(0).round(4)

    df_final = pd.concat([df, df_concepts], axis=1)

    return df_final

In [71]:
def concatenar_arquivos_parquet(folder_path):
    dataframes = []

    for filename in os.listdir(folder_path):
        if filename.endswith('.parquet'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_parquet(file_path)
            dataframes.append(df)

    df_concatenado = pd.concat(dataframes, ignore_index=True)
    
    df_concatenado.to_parquet('df_concatenado.parquet', index=False)

In [88]:
def processar_dataframe(df):
    df = df.loc[:, ['doi', 'title', 'publication_date', 'open_access', 'concepts']]
    
    df['open_access'] = df['open_access'].apply(lambda x: x['is_oa'])

    df = extrair_concepts_scores(df)

    return df

In [67]:
coletar_dados_e_salvar('2023-10-04','2023-10-04')

Total de erros: 0


In [90]:
df = pd.read_parquet('/home/franciscofoz/Documents/GitHub/recomendador-artigos-OpenAlex-GPT/datasets_2023-10-04_to_2023-10-04/registros_1.parquet')


In [91]:
df_processado = processar_dataframe(df)

In [101]:
for coluna in df_processado.iloc[:,5:].columns:
    if df_processado[coluna].gt(0).any():
        print(coluna)

Geography
Biology
Medicine
Sociology
Chemistry
Physics
Engineering
Philosophy
Psychology
Art
Mathematics
Computer science
Materials science
Business
Political science
History


In [106]:
def extrair_concepts(df):
    concepts = []

    for coluna in df_processado.iloc[:,5:].columns:
        if df_processado[coluna].gt(0).any():
            concepts.append(coluna)

    df['concepts'] = df.apply(lambda row: ';'.join([col for col in concepts if row[col] > 0]), axis=1)

    return df


In [107]:
extrair_concepts(df_processado)

,doi,title,publication_date,open_access,concepts,Geography,Biology,Medicine,Sociology,Chemistry,Physics,Engineering,Philosophy,Psychology,Art,Mathematics,Computer science,Materials science,Economics,Business,Political science,History
0,https://doi.org/10.24425/pjvs.2018.125598,Four years of African swine fever in Poland. N...,2023-10-04,True,Geography;Biology;Medicine,0.3814,0.3451,0.1766,0.0,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,https://doi.org/10.24425/pjvs.2019.131407,Virucidal effect of chosen disinfectants again...,2023-10-04,True,Medicine;Chemistry,0.0000,0.0000,0.2281,0.0,0.5065,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,https://doi.org/10.24425/pjvs.2018.125597,Comparison of loop-mediated isothermal amplifi...,2023-10-04,True,Biology;Medicine;Physics,0.0000,0.3465,0.3522,0.0,0.0000,0.0651,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,https://doi.org/10.24425/pjvs.2019.129226,Effects of porcine epidemic diarrhea virus inf...,2023-10-04,True,Biology;Medicine,0.0000,0.6963,0.1177,0.0,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,https://doi.org/10.24425/pjvs.2019.131415,Vertical transmission of anti-ASFV antibodies ...,2023-10-04,True,Biology;Medicine;Engineering,0.0000,0.4344,0.2577,0.0,0.0000,0.0000,0.053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,https://doi.org/10.24425/pjvs.2021.139982,Effect of kebar grass extract ( Biophytum pete...,2023-10-04,True,Biology;Medicine,0.0000,0.4380,0.2781,0.0,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
196,https://doi.org/10.24425/pjvs.2022.140843,The bacteriostatic effect of erythritol on can...,2023-10-04,True,Biology;Medicine,0.0000,0.3344,0.4414,0.0,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
197,https://doi.org/10.24425/pjvs.2022.140839,S1 gene based phylogeny of Israel variant-2 in...,2023-10-04,True,Biology;Medicine,0.0000,0.5429,0.2642,0.0,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
198,https://doi.org/10.24425/pjvs.2022.140842,Evaluation of a hydrolysed salmon and pea hypo...,2023-10-04,True,Medicine,0.0000,0.0000,0.6944,0.0,0.0000,0.0000,0.000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
